In [ ]:
import openai
import pandas as pd
import random
import os
import json

openai.api_key = "sk-pA"

ability_dict = {
    "CR": "Causal Reasoning",
    "CT": "Creativity",
    "ICL": "In-context Learning",
    "IF": "Instruction Following",
    "MT": "Machine Translation",
    "SUMM": "Summarization",
}

inspiration_prompt_template = """
I want you to take on the role of an ingenious and imaginative assistant. Your mission is to spark creativity and inspire innovative thinking. For the given topic of {capability} ability, please provide a truly unique and thought-provoking inspiration sentence. 

This inspiration should:
1. Challenge conventional thinking and encourage thinking outside the box.
2. Stimulate a wide array of creative and critical ideas.
3. Serve as a catalyst for generating engaging and meaningful tasks or questions.

Be bold, imaginative, and daring in your approach. Let's create something extraordinary!
"""

prompt_template = """
The goal of this task is to evaluate the model's ability to demonstrate {capability}. Imagine you are designing a challenging yet creative evaluation question. Using the example provided below as a guide and the inspiration as a springboard, craft a new and exciting question that captures the essence of this capability. 

Make the question engaging, thought-provoking, and appropriately complex, while ensuring it evaluates the desired skill effectively. Generate a JSON object with the following fields: question (string), reference (string).


Here are the inputs:
- Example: {example}
- Inspiration: {inspiration}

Let your imagination flow and create a question that stands out!
"""

output_dir = "/home/lujun_li/projects/temperature_eval/data/Augemented/"
os.makedirs(output_dir, exist_ok=True)

for ability_code, capability_name in ability_dict.items():
    try:
        target_df = pd.read_csv(
            f"/home/lujun_li/projects/temperature_eval/data/Intermediate/{ability_code}.csv"
        )

    except (FileNotFoundError, KeyError):
        print(
            f"Warning: File for {ability_code} not found or invalid format, skipping."
        )
        continue

    generated_data = []

    if ability_code == "MT":
        repetition = 100
    if ability_code == "CT":
        repetition = 10
    else:
        repetition = 50
    for _ in repetition(repetition):
        random_row = target_df.sample(n=1, random_state=random.randint(0, 100)).iloc[0]
        example = random_row["input"]  

        inspiration_prompt = inspiration_prompt_template.format(
            capability=capability_name
        )
        inspiration_response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": inspiration_prompt},
            ],
            temperature=0.7,  
        )
        inspiration = inspiration_response.choices[0].message.content.strip()

        prompt = prompt_template.format(
            capability=capability_name,
            example=example,
            inspiration=inspiration,
        )
        question_response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": "You are an assistant that only outputs JSON-formatted data. No extra explanations, no text outside JSON, and strict adherence to the required format.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.7, 
        )
        question_json = question_response.choices[0].message.content.strip()

        try:
            question_json = json.loads(question_json)
            question = question_json.get("question", "")
            reference = question_json.get("reference", "")
        except json.JSONDecodeError:
            print(f"Invalid JSON format: {question_json}")
            question = ""
            reference = ""
        generated_data.append(
            {
                "Ability": capability_name,
                "Example": example,
                "Inspiration": inspiration,
                "Question": question,
                "Reference": reference,
            }
        )

        print(f"Ability: {capability_name}")
        print(f"Inspiration: {inspiration}")
        print(f"Generated Question: {question_json}")
        print("-" * 50)

    output_file = os.path.join(output_dir, f"{ability_code}_generated_questions.csv")
    pd.DataFrame(generated_data).to_csv(output_file, index=False, encoding="utf-8")
    print(f"Saved 50 questions for {capability_name} to {output_file}.")

In [ ]:
import pandas as pd

CR_df = pd.read_csv(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Augemented/CR_generated_questions.csv"
)

In [ ]:
CT_df = pd.read_csv(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Augemented/CT_generated_questions.csv"
)

In [ ]:
ICL_df = pd.read_csv(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Augemented/ICL_generated_questions.csv"
)

In [19]:
import openai
import json
from tqdm import tqdm

file_path = "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/.vscode/api_key.txt"
with open(file_path, "r") as file:
    api_key = file.read().strip()
openai.api_key = api_key

IF_df = pd.read_csv(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Augemented/IF_generated_questions.csv"
)

prompt_template = """I would like you to generate three key instruction verification questions based on the following input question, which I asked the model:

{question}

These questions should assess the model's ability to accurately follow the instructions provided in the initial question. Please return the three important questions as a JSON object with the following fields:

"Q1" (string): The first key question to verify the model's ability to follow the instruction.
"Q2" (string): The second key question to verify the model's ability to follow the instruction.
"Q3" (string): The third key question to verify the model's ability to follow the instruction.
Ensure that the questions are directly relevant to the instruction and are crucial for evaluating the model’s performance in following the given task.
"""

with open(
    "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Augemented/output.jsonl",
    "a",
) as jsonl_file:
    # Wrap the iteration with tqdm to display a progress bar
    for i, row in tqdm(IF_df.iterrows(), total=IF_df.shape[0], desc="Processing Rows"):
        update_row = row.copy()
        question = row["Question"]
        prompt = prompt_template.format(question=question)

        try:
            inspiration_response = (
                openai.chat.completions.create(
                    model="gpt-4",
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": prompt},
                    ],
                    temperature=0.7,  # Adjust randomness
                )
                .choices[0]
                .message.content.strip()
            )

            # Attempt to parse the JSON response
            question_json = json.loads(inspiration_response)
            Q1 = question_json.get("Q1", "")
            Q2 = question_json.get("Q2", "")
            Q3 = question_json.get("Q3", "")
            Qs = [Q1, Q2, Q3]

            update_row["questions_to_be_followed"] = str(Qs)

        except json.JSONDecodeError:
            print(f"Invalid JSON format for question: {question}")
            Qs = ["", "", ""]
            update_row["questions_to_be_followed"] = str(
                Qs
            )  # Store empty list if error occurs

        except Exception as e:
            print(f"Error processing question: {question}. Error: {e}")
            Qs = ["", "", ""]
            update_row["questions_to_be_followed"] = str(
                Qs
            )  # Store empty list in case of other errors

        # Write the updated row as a JSON object to the jsonl file
        jsonl_file.write(json.dumps(update_row.to_dict()) + "\n")

Processing Rows:  98%|█████████▊| 49/50 [03:03<00:03,  3.74s/it]

Invalid JSON format for question: nan


Processing Rows: 100%|██████████| 50/50 [03:06<00:00,  3.73s/it]
